In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%time
from baseline import main, main_new
main()
#main_new()

In [ ]:
%%time
from make_submission import main
main()
#target, predictions_prob, y_pred_prob = main()

In [ ]:
from collections import Counter

# Предположим, что y_val - это массив меток классов
class_balance = Counter(predictions)
print("Баланс классов:", class_balance)

In [ ]:
%%time
from baseline import make_report
make_report()

In [ ]:
%%time
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'start.ipynb'
from baseline import train_bert, check_bert
import wandb
wandb.login(key="68be83ff5b81233c60ffad40f6327a3700797a16")
check_bert()
#train_bert()

In [ ]:
%%time
from baseline import make_report
make_report()

In [ ]:
import numpy as np
text_data = train_data['text_1'] + ' ' + train_data['text_2']    
text_embeddings = tfidf_vectorizer.transform(text_data).toarray()
train_data['combined_embeddings'] = train_data.apply(lambda row: np.concatenate([
        row['pic_embeddings_1'][0], row['pic_embeddings_2'][0], text_embeddings[row.name]
    ]), axis=1)

In [ ]:
import pyarrow.parquet as pq
import pandas as pd

attributes_path = './data/test/attributes_test.parquet'
resnet_path = './data/test/resnet_test.parquet'
text_and_bert_path = './data/test/text_and_bert_test.parquet'
val_path = './data/test/test.parquet'

test = pd.read_parquet(val_path, engine='pyarrow')
variantids = pd.concat([test['variantid1'], test['variantid2']]).unique()
# Открываем файл
parquet_file = pq.ParquetFile(resnet_path)

# Задаем пустой DataFrame для хранения отфильтрованных данных
filtered_df = pd.DataFrame()

# Обрабатываем файл по row group
# for i in range(parquet_file.num_row_groups):
i = 0
for batch in parquet_file.iter_batches():

    i += 1
    print(i)
    
    batch = batch.to_pandas()
    
    # # Применяем фильтрацию
    filtered_df = pd.concat([filtered_df, batch[batch['variantid'].isin(variantids)]], ignore_index=True)

# Используем отфильтрованные данные
# text_and_bert = filtered_df

In [ ]:
import numpy
print(numpy.__version__)

In [ ]:
text_data.loc[0]

In [ ]:
text_embeddings[0]

In [ ]:
attributes.loc[2]

In [ ]:
attributes.loc[2]['categories']

In [ ]:
attributes.loc[2]['characteristic_attributes_mapping']

In [ ]:
attributes.loc[0]['characteristic_attributes_mapping']

In [ ]:
attributes.loc[2]['combined_text']

In [ ]:
train_data.columns

In [ ]:
train_data_filtered.shape

In [ ]:
train_data.columns

In [ ]:
val_data.columns

In [ ]:
train_data.loc[0]

In [ ]:
val_data.loc[0]

In [ ]:
text_and_bert.loc[0]['name_bert_64']

In [ ]:
attributes_filtered.shape

In [ ]:
attributes.shape

In [ ]:
text_and_bert.columns

In [ ]:
text_and_bert.head()

In [ ]:
attributes.columns

In [ ]:
attributes.head()

In [ ]:
attributes.loc[2]['characteristic_attributes_mapping']

In [ ]:
text_and_bert.loc[0]['name']

In [ ]:
text_and_bert.loc[0]['description']

In [ ]:
text_and_bert.columns

In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
%%time
from baseline import process_text_and_bert
attributes = process_text_and_bert(attributes)

In [ ]:
%%time
from baseline import merge_data
train_data = merge_data(attributes, train, resnet, text_and_bert)

In [ ]:
import joblib
X_val = joblib.load('X_val.pkl')

In [ ]:
X_val.shape

In [ ]:
# import joblib
# X_val = joblib.load('X_val.pkl')
# y_val = joblib.load('y_val.pkl')
# tfidf_vectorizer = joblib.load('vectorizer.pkl')
# model = joblib.load('baseline.pkl')
# y_pred_prob = model.predict_proba(X_val)[:, 1]
# y_pred = (y_pred_prob >= 0.5).astype(int)
# y_pred[:20]

In [ ]:
%%time
from make_submission import main
main()
#target, predictions_prob, y_pred_prob = main()

### Combined predictions

In [ ]:
import numpy as np
predictions = (predictions_prob >= 0.5).astype(int)
y_pred_prob = np.array(y_pred_prob)
y_pred = (y_pred_prob >= 0.5).astype(int)

In [ ]:
from baseline import evaluate_predictions
evaluate_predictions(target, y_pred, y_pred_prob)

In [ ]:
evaluate_predictions(target, predictions, predictions_prob)

In [ ]:
combined_prob = (predictions_prob*0.6 + y_pred_prob*0.4)
combined_predictions = (combined_prob >= 0.5).astype(int)
evaluate_predictions(target, combined_predictions, combined_prob)

### check inferens

In [ ]:
%%time
from make_submission import main
test_data = main()
text_1_list = test_data['text_1'].astype(str).tolist()
text_2_list = test_data['text_2'].astype(str).tolist()
del test_data

In [ ]:
%%time
import joblib
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
model = torch.load('bert.pkl')
tokenizer = joblib.load("tokenizer.pkl")
inputs = tokenizer(text_1_list, text_2_list, padding=True, truncation=True, return_tensors="pt", max_length=256)

In [ ]:
%%time
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'])
dataloader = DataLoader(dataset, batch_size=1)
# Массивы для хранения предсказаний
all_preds = []
all_probs = []

In [ ]:
%%time
device="cpu"
model.eval()
with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_mask, token_type_ids = [x.to(device) for x in batch]
        break

In [ ]:
%%time
outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
logits = outputs.logits
probs = torch.softmax(logits, dim=1)
preds = torch.argmax(probs, dim=1)

In [ ]:
all_preds.extend(preds.cpu().detach().numpy())
all_probs.extend(probs[:, 1].cpu().detach().numpy())

In [ ]:
type(preds)

In [ ]:
from pympler import asizeof
from transformers import AutoTokenizer

# Пример с токенизацией
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# inputs = tokenizer("Hello, how are you?", return_tensors="pt")

# Получение размера объекта BatchEncoding
size = asizeof.asizeof(inputs['input_ids'])

print(f"Размер BatchEncoding в памяти: {size} байт")

In [ ]:
inputs['input_ids'].shape

In [ ]:
from baseline import save_trained_model
save_trained_model()

In [ ]:
import joblib
# train_texts = joblib.load('./data/train_texts.pkl')
train_texts = joblib.load('./data/val_texts.pkl')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertModel
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch

MAX_TOKENS=1024

# Допустим, у вас есть такие переменные
# train_texts - список текстов
# tokenizer - ваш токенайзер
# MAX_TOKENS - максимальная длина

tokenizer = BertTokenizer.from_pretrained('rubert-tiny2')

# Сначала токенизируем все тексты и определяем их длину
token_lengths = [len(tokenizer.encode(text, truncation=False)) for text in train_texts]

# Определяем, сколько текстов превышают MAX_TOKENS
num_exceeding = sum(1 for length in token_lengths if length > MAX_TOKENS)

# Вычисляем процент
percent_exceeding = (num_exceeding / len(train_texts)) * 100

print(f"Процент данных, длина которых превышает {MAX_TOKENS} токенов: {percent_exceeding:.2f}%")

In [ ]:
type(train_texts)

In [ ]:
train_texts = train_texts[:100]

In [ ]:
arr = type(train_texts)

In [ ]:
list1 = [1, 2, 3]
list2 = [4, 5, 6]
list3 = list1 + list2

print(list3)